# Simple Stock Prediction Tensorflow

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AAPL'
start = '2018-01-01'
end = '2019-01-31'

# Read data 
df = yf.download(symbol,start,end)

# View Columns
df.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,170.160004,172.300003,169.259995,172.259995,168.987320,25555900
2018-01-03,172.529999,174.550003,171.960007,172.229996,168.957886,29517900
2018-01-04,172.539993,173.470001,172.080002,173.029999,169.742706,22434600
2018-01-05,173.440002,175.369995,173.050003,175.000000,171.675278,23660000
2018-01-08,174.350006,175.610001,173.929993,174.350006,171.037628,20567800


In [3]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-25,155.479996,158.130005,154.320007,157.759995,157.086288,33535500
2019-01-28,155.789993,156.330002,153.660004,156.300003,155.632523,26192100
2019-01-29,156.250000,158.130005,154.110001,154.679993,154.019440,41587200
2019-01-30,163.250000,166.149994,160.229996,165.250000,164.544296,61109800
2019-01-31,166.110001,169.000000,164.559998,166.440002,165.729218,40739600


In [4]:
# Create Data
df['Open_Close'] = (df['Open'] - df['Adj Close'])/df['Open']
df['High_Low'] = (df['High'] - df['Low'])/df['Low']
df['Increase_Decrease'] = np.where(df['Volume'].shift(-1) > df['Volume'],1,0)
df['Buy_Sell_on_Open'] = np.where(df['Open'].shift(-1) > df['Open'],1,0)
df['Buy_Sell'] = np.where(df['Adj Close'].shift(-1) > df['Adj Close'],1,0)
df['Returns'] = df['Adj Close'].pct_change()
df = df.dropna()
df.head()

,Open,High,Low,Close,Adj Close,Volume,Open_Close,High_Low,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,,,
2018-01-03,172.529999,174.550003,171.960007,172.229996,168.957886,29517900,0.020704,0.015062,0,1,1,-0.000174
2018-01-04,172.539993,173.470001,172.080002,173.029999,169.742706,22434600,0.016212,0.008078,1,1,1,0.004645
2018-01-05,173.440002,175.369995,173.050003,175.000000,171.675278,23660000,0.010175,0.013406,0,1,0,0.011385
2018-01-08,174.350006,175.610001,173.929993,174.350006,171.037628,20567800,0.018998,0.009659,1,1,0,-0.003714
2018-01-09,174.550003,175.059998,173.410004,174.330002,171.018005,21584000,0.020235,0.009515,1,0,0,-0.000115


In [5]:
df.shape

(271, 12)

Dense is a layer of connected neurons.

Loss Function is used to optimize the parameter values in a neural network model. Therefore, it measures squared or absolute error between a network's output and some target or desired output.

Optimizer generates a new and improved values.

In [6]:
import tensorflow as tf
from tensorflow import keras

In [7]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')

In [8]:
X = np.array(df['Returns'], dtype = float) # Feature
Y = np.array(df['Adj Close'], dtype = float) # Target

In [9]:
model.fit(X, Y, epochs=100)

Epoch 1/100
271/271 [==============================] - 0s 384us/step - loss: 29862.6151
Epoch 2/100
271/271 [==============================] - 0s 18us/step - loss: 20882.1033
Epoch 3/100
271/271 [==============================] - 0s 22us/step - loss: 14649.7054
Epoch 4/100
271/271 [==============================] - 0s 18us/step - loss: 10321.6915
Epoch 5/100
271/271 [==============================] - 0s 18us/step - loss: 7311.5090
Epoch 6/100
271/271 [==============================] - 0s 18us/step - loss: 5232.4851
Epoch 7/100
271/271 [==============================] - 0s 18us/step - loss: 3778.5427

In [10]:
print(model.predict([0])) # Predict on open at 18.15, What the closing price prediction?

[[ 184.52786255]]


If the values return nan.  Therefore, we need to normalize the data.

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_X = scaler.fit_transform(np.array(df['Returns']).reshape(271,-1))
X = np.array(normalized_X, dtype = float)
normalized_Y = scaler.fit_transform(np.array(df['Adj Close']).reshape(271,-1))
Y = np.array(normalized_Y, dtype = float)

In [12]:
print(X.shape)
print(Y.shape)

(271, 1)
(271, 1)


In [13]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [14]:
# Still have nan
print(model.predict([0])) # Predict on open at 18.15, What the closing price prediction?

[[ 184.52786255]]
